In [1]:
from utility_common import *
from utility_plotter import *
#import ROOT
from IPython.display import clear_output
## add plotting option and load plotting parameter table
%matplotlib inline
#%qtconsole --style monokai
#plt.style.use('classic')

plt.rc("font",size=10)
plt.rc("savefig",dpi=300)
plt.rc("figure",facecolor="w",figsize=(6,4))


In [8]:
def getPickdir(selection):
    pickledir  =  "/mnt/data/zchen/Analysis/pickle/{}/".format(selection)
    if selection is "emu2":
        pickledir  =  "/mnt/data/zchen/Analysis/pickle/emu/"
    return pickledir

def getPP(selection):
    if selection in ["e4j","mu4j"]:
        pp = pd.read_csv("plotparameters4j.csv")
    else:
        pp = pd.read_csv("plotparameters.csv")
    return pp

def getWeightAdjustName(name):
    variables = {"RenormUp":"qcd_weight_up_nom",
                 "RenormDown":"qcd_weight_down_nom",
                 "FactorUp":"qcd_weight_nom_up",
                 "FactorDown":"qcd_weight_nom_down",
                 "PDFUp":"pdf_weight_up",
                 "PDFDown":"pdf_weight_down"}
    return variables[name]

    
def makePlot(selection,nbjetcut):
    
    pickledir = getPickdir(selection)
    pp        = getPP(selection)
    
    cuts = GetSelectionCut(selection,nbjetcut)
    MC   = LoadDataframe(pickledir + "mctt").query(cuts)
    
    MC["pdf_weight_up"]   = 1+np.sqrt(MC.pdf_var/99)
    MC["pdf_weight_down"] = 1-np.sqrt(MC.pdf_var/99)

    for index, row in pp.iterrows():
        v,a,b,step,xl = row["var"],row["lower"],row["upper"],row["step"],row["xlabel"]
        mybins = np.arange(a,b,step)
        mybinscenter = mybins[0:-1]+step/2

        fig, axes = plt.subplots(2, 1, sharex=True, facecolor="w",
                                 gridspec_kw={'height_ratios':[2,1]},
                                 figsize=(6,6))
        fig.subplots_adjust(hspace=0)

        ax   = axes[0] 
        cent = ax.hist(MC[v],weights=MC["eventWeight"],bins=mybins,histtype="step",color='k',alpha=0,lw=1,label=None)[0]

        for iname,name in enumerate(["Renorm","Factor","PDF"]):
            
            ax   = axes[0] 
            up0  = np.histogram(MC[v],  weights=MC["eventWeight"]*MC[getWeightAdjustName(name+"Up")], bins=mybins)[0]
            down0= np.histogram(MC[v],  weights=MC["eventWeight"]*MC[getWeightAdjustName(name+"Down")], bins=mybins)[0]
            up   = (up0+down0)/2+np.abs(up0-down0)/2
            down = (up0+down0)/2-np.abs(up0-down0)/2
            ax.bar(mybins[:-1]+step/2, up-down, step,bottom=down,color="C{}".format(iname),label=name,alpha=0.4)
            ax.legend(fontsize=14)
            ax.grid(True,linestyle="--",alpha=0.5)

            ax = axes[1]
            nonzero = cent>0
            downratio = down[nonzero]/(cent[nonzero])
            upratio = up[nonzero]/cent[nonzero]
            ax.fill_between(mybinscenter[nonzero],upratio,downratio,color="C{}".format(iname),alpha=0.4)


        ax = axes[0]
        cent = ax.hist(MC[v],weights=MC["eventWeight"],bins=mybins,histtype="step",color='k',alpha=1,lw=1,label="nominal")[0]
        ax.legend(fontsize=14)
        

        ax = axes[1]
        ax.axhline(1,color="k")
        ax.set_ylim(0.8,1.2)
        ax.grid(True,linestyle="--",alpha=0.5)

        ax.set_xlabel(xl,fontsize=12)
        
        plt.savefig("../../plot/{}/ttTheory2/{}.png".format(selection,v),dpi=300)
        
        print("making plots nbjet{0}: {1}/{2}".format(nbjetcut,index,len(pp)))
        clear_output(wait=True)
        plt.close()


In [9]:
for selection in ["emu","mumu","mutau","ee","emu2","etau","mu4j","e4j"]:
    for nbjetcut in ["==1",">1"]:
        makePlot(selection,nbjetcut)

making plots nbjet>1: 23/24
